# Data Science Peer Graded Assignment - Week 3


In [2]:
!pip install pgeocode
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
done

# All requested packages already installed.



In [5]:
conda update --all

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1976
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1976
failed
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                 

In [4]:
!pip uninstall -y numpy
!pip uninstall -y pandas

Found existing installation: numpy 1.20.2
Uninstalling numpy-1.20.2:
  Successfully uninstalled numpy-1.20.2
Found existing installation: pandas 1.2.3
Uninstalling pandas-1.2.3:
  Successfully uninstalled pandas-1.2.3


In [5]:

!pip install pandas
!pip install numpy

  Using cached pandas-1.2.3-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-watson-machine-learning 1.0.57 requires pandas<=1.0.5, but you have pandas 1.2.3 which is incompatible.


In [34]:
#import libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pgeocode
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium 

## Loading the dataset from webpage

In [7]:
Toronto_text=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(Toronto_text,'html.parser')

## Storing the data into a DataFrame

In [8]:
table_contents=[]
table = soup.find('table') #read the table using BeautifulSoup object from the webpage

for row in table.findAll('td'):  #for each row read the values of cols
    cell = {}
    if row.span.text =='Not assigned': #check if the postalcode is assigned, ignore if not
        pass
    else: # read postalcode, borough and neighborhood
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

## Loading the latitude and longitude and appending to dataset df

In [9]:
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist() #to list
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    g = geolocator.query_postal_code(postal_code)
    if not g.empty:
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

df['Latitude'] = latitudes
df['Longitude'] = longitudes

#drop any Nan values from latitude and longitude
df = df.dropna(subset=['Latitude'])
df = df.dropna(subset=['Longitude'])

df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto Business,Enclave of M4L,43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


In [10]:
# no. of boroughs and neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 14 boroughs and 102 neighborhoods.


In [11]:
# Getting the geographical address of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Using folium, illustrate the points on map

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, pc in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df['PostalCode']):
    label = '{}, {}'.format(neighborhood, borough, pc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Extract dataset of North York's neighborhood

In [14]:
northyork_data = df[df['Borough'] == 'North York'].reset_index(drop=True)
northyork_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
3,M3B,North York,Don Mills North,43.7450,-79.3590
4,M6B,North York,Glencairn,43.7081,-79.4479
5,M3C,North York,Don Mills South,43.7334,-79.3329
6,M2H,North York,Hillcrest Village,43.8015,-79.3577
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.7801,-79.3479
9,M3J,North York,"Northwood Park, York University",43.7694,-79.4921


## Just to explore ;)

In [15]:
queenspark_data = df[df['Borough']=='Queen\'s Park'].reset_index(drop=True)
queenspark_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889


## get geographical coordinates of North Park, Ca

In [16]:
address = 'North Park, Toronto, ca'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7186899, -79.4775337.


In [17]:
#illustrate the map
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

# Foursquare details to generate an url and further analyse the neighborhoods

In [18]:
CLIENT_ID = 'UWC5CWZ5DD3244YCUQ3XKPDCAPG3LS1U5EWAAITZLEYC0Z3Q' # your Foursquare ID
CLIENT_SECRET = 'LATZ4S5ITZWDBV110CE1P5DLBXZI3ADFPUGONNOH25WVQVJ1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [19]:
northyork_data.loc[0, 'Neighborhood']

'Parkwoods'

In [20]:
#details required to generate an url
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7545, -79.33.


In [21]:
#Generate URL and store the results in the json
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()


In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
nearby_venues =[]
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply([get_category_type], axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Getting all the nearby venues (from lab)

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
#North York nearby venues
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  ) 


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [27]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",7,7,7,7,7,7
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills North,4,4,4,4,4,4
Don Mills South,4,4,4,4,4,4
Downsview Central,2,2,2,2,2,2
Downsview East,5,5,5,5,5,5
Downsview Northwest,19,19,19,19,19,19
Downsview West,3,3,3,3,3,3


In [28]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 113 uniques categories.


In [29]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Baby Store,Bakery,Bank,Bar,...,Sports Bar,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#Calculate the mean of the distances
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Baby Store,Bakery,Bank,Bar,...,Sports Bar,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.040000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.040000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills North,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills South,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview East,0.000000,0.2,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview Northwest,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000
8,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairview, Henry Farm, Oriole",0.000000,0.0,0.015625,0.000000,0.015625,0.00,0.015625,0.031250,0.015625,...,0.0,0.000000,0.015625,0.00,0.015625,0.031250,0.000000,0.015625,0.000000,0.000000


## Top 5 venues in the vicinity of North York

In [31]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0        Fried Chicken Joint  0.14
1              Deli / Bodega  0.14
2                Coffee Shop  0.14
3              Grocery Store  0.14
4  Middle Eastern Restaurant  0.14


----Bayview Village----
                      venue  freq
0               Gas Station  0.25
1                      Park  0.25
2                     Trail  0.25
3               Flower Shop  0.25
4  Mediterranean Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1         Coffee Shop  0.08
2  Italian Restaurant  0.08
3      Breakfast Spot  0.04
4    Greek Restaurant  0.04


----Don Mills North----
                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2                        Pool  0.25
3      Furniture / Home Store  0.25
4                  Kids Store  0.00


----Don Mills South----
               venue  fr

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Fried Chicken Joint,Deli / Bodega,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Mediterranean Restaurant,Pizza Place,Accessories Store,Pet Store,Park
1,Bayview Village,Gas Station,Park,Trail,Flower Shop,Mediterranean Restaurant,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Breakfast Spot,Greek Restaurant,Restaurant,Grocery Store,Pub,Café,Butcher
3,Don Mills North,Construction & Landscaping,Park,Pool,Furniture / Home Store,Kids Store,Mexican Restaurant,Pharmacy,Pet Store,Juice Bar,Karaoke Bar
4,Don Mills South,Park,Trail,Gym,River,Accessories Store,Mediterranean Restaurant,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop


KMea

In [35]:

# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 1, 1, 1, 1, 4, 1], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [37]:
northyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0,Park,Food & Drink Shop,Accessories Store,Pizza Place,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
1,M4A,North York,Victoria Village,43.7276,-79.3148,1,Intersection,Coffee Shop,Park,Portuguese Restaurant,Hockey Arena,Pizza Place,Metro Station,Pet Store,Nightclub,Moving Target
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,1,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sushi Restaurant,Toy / Game Store,Bakery,Juice Bar,Sandwich Place,Men's Store
3,M3B,North York,Don Mills North,43.7450,-79.3590,4,Construction & Landscaping,Park,Pool,Furniture / Home Store,Kids Store,Mexican Restaurant,Pharmacy,Pet Store,Juice Bar,Karaoke Bar
4,M6B,North York,Glencairn,43.7081,-79.4479,1,Pizza Place,Latin American Restaurant,Fast Food Restaurant,Gas Station,Mediterranean Restaurant,Grocery Store,Sushi Restaurant,Japanese Restaurant,Bakery,Ice Cream Shop


In [38]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Accessories Store,Pizza Place,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
6,North York,0,Park,Residential Building (Apartment / Condo),Accessories Store,Pizza Place,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
22,North York,0,Park,Convenience Store,Daycare,Accessories Store,Pharmacy,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop


In [40]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Intersection,Coffee Shop,Park,Portuguese Restaurant,Hockey Arena,Pizza Place,Metro Station,Pet Store,Nightclub,Moving Target
2,North York,1,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sushi Restaurant,Toy / Game Store,Bakery,Juice Bar,Sandwich Place,Men's Store
4,North York,1,Pizza Place,Latin American Restaurant,Fast Food Restaurant,Gas Station,Mediterranean Restaurant,Grocery Store,Sushi Restaurant,Japanese Restaurant,Bakery,Ice Cream Shop
5,North York,1,Park,Trail,Gym,River,Accessories Store,Mediterranean Restaurant,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop
7,North York,1,Fried Chicken Joint,Deli / Bodega,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Mediterranean Restaurant,Pizza Place,Accessories Store,Pet Store,Park
8,North York,1,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Coffee Shop,Mobile Phone Shop,Bank,Baseball Field,Toy / Game Store,Juice Bar
9,North York,1,Massage Studio,Middle Eastern Restaurant,Sports Bar,Sandwich Place,Pizza Place,Men's Store,Pet Store,Park,Nightclub,Moving Target
10,North York,1,Gas Station,Park,Trail,Flower Shop,Mediterranean Restaurant,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop
11,North York,1,Shoe Store,Airport,Park,Coffee Shop,Food Court,Men's Store,Pet Store,Nightclub,Moving Target,Movie Theater
14,North York,1,Trail,Bakery,Basketball Court,Accessories Store,Men's Store,Pharmacy,Pet Store,Park,Nightclub,Moving Target


In [41]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,2,Playground,Seafood Restaurant,Accessories Store,Men's Store,Pet Store,Park,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop


In [42]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,3,Pool,Accessories Store,Pizza Place,Pet Store,Park,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop


In [43]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4,Construction & Landscaping,Park,Pool,Furniture / Home Store,Kids Store,Mexican Restaurant,Pharmacy,Pet Store,Juice Bar,Karaoke Bar
13,North York,4,Pool,Park,Bus Stop,Accessories Store,Men's Store,Pet Store,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop
15,North York,4,Furniture / Home Store,Home Service,Mediterranean Restaurant,Pet Store,Park,Nightclub,Moving Target,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
